#### Prepare MitoFish data

MitoFish website: http://mitofish.aori.u-tokyo.ac.jp/download.html

##### Download and process mitoannotator (complete mitogenome) records:

```
# Download and unzip file in a folder named 'mitoannotations'
mkdir mitoannotations
cd mitoannotations
wget http://mitofish.aori.u-tokyo.ac.jp/files/mitoannotations.zip
unzip mitoannotations.zip

# get accession numbers
ls *.txt | cut -d '_' -f1,2 >complete.accession

# get species names
ls *.txt | cut -d '_' -f3- | sed "s/.txt/#complete mitogenomes/g" >complete.species

# Make a list of accession number and species, separated by "#"
paste -d "#" complete.accession complete.species >complete.list

```
##### Download and process complete+partial mDNA sequence file: 

```
# Download and unzip sequence file in the same "mitoannotations" folder
wget http://mitofish.aori.u-tokyo.ac.jp/files/complete_partial_mitogenomes.zip
unzip complete_partial_mitogenomes.zip

# get accession numbers
grep ">" complete_partial_mitogenomes.fa | awk -F "|" '{print $2}' >mitofish.accession
```

#### Prepare NCBI data

NCBI blast databases ftp site: https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/

```
# Download and unzip NCBI sequence file in the "NCBI" subfolder
mkdir NCBI
cd NCBI
wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nt.gz
wget https://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nt.gz.md5
md5sum -c nt.gz.md5 >md5sum.log
unzip nt.gz

# get accession numbers
grep ">" nt | cut -d ' ' -f1 | tr -d ">"  >nt.accession 

# get gene names
grep ">" nt | cut -d ' ' -f2- >nt.genenames 

# Make a list of accession number and species, separated by "#"
paste -d "#" nt.accession nt.genenames >nt.list
cd ..
```

#### Extract (grep) 12S genes from NCBI records

```
grep -e "12S ribosomal" -e "12S rRNA" nt.list >12S.list
```

In [162]:
# This python script creates a dictionary from the nt.list file
# The key is the NCBI accession number and the value is the gene description
# This dictionary is then saved as a pickle named nt.pickle

#!/usr/bin/python
import pickle
ntdict = {'Accession':'Gene description'}

with open("nt.list",'r') as f:
    for line in f:
        line = line.rstrip()
        entry = line.split("#")
        fullacc=entry[0].split(".")
        newentry={fullacc[0]:entry[1]} 
        ntdict.update(newentry)

with open("nt.pickle", 'wb') as handle:
    pickle.dump(ntdict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [2]:
# This python script loads nt.pickle 
# Then, it matches MitoFish entries with keys (NCBI accession numbers) in the pickle
# It takes ~24 minutes to match 616,999 MitoFish records to 57,377,397 NCBI GenBank records
# cpupercent=101,cput=00:22:53,mem=34500088kb,ncpus=24,vmem=34845840kb,walltime=00:23:34

#!/usr/bin/python

import pickle

import sys
import os.path
from os import path

with open("nt.pickle", 'rb') as handle:
    NCBI = pickle.load(handle)

outfile="mitofish.genes"
nohit=0
count=0
length=(len(NCBI))

if path.exists(outfile) :
    sys.exit("Error: Output file exists! Please rename output file and try again!")

output=open(outfile,'a')

print("==== Searching... ====")

with open("mitofish.accession",'r') as infile:
    for inline in infile:
        inline = inline.rstrip()
        count +=1
        if inline in NCBI:
            output.write("%s#%s\n" % (inline,NCBI[inline]))
        elif inline not in NCBI:
            output.write("%s#No hit found!\n" % inline)
            print("%s#No hit found!" % inline)
            nohit +=1
            
output.close()

print ("==== Run complete! ===")
print ("Total: %d accession numbers" % count)
print ("No hits for %d input accession numbers!" % nohit)


==== Searching... ====
SVSP109#No hit found!
alpha-lactalbumin#No hit found!
LN610210#No hit found!
LN610216#No hit found!
miami#No hit found!
==== Run complete! ===
Total: 11 accession numbers
No hits for 5 input accession numbers!


### Pick up "missing" accession numbers 

In NCBI fasta files, records with duplicate sequences will be concatenated in the header line, e.g.

```
>LN610233.1 Chiloglanis anoterus mitochondrial partial D-loop, specimen voucher 68321_34LN610234.1 Chiloglanis anoterus mitochondrial partial D-loop, specimen voucher 68330_55
```
These will not be picked up by the python script above. First, let's extract the accession numbers with no hits into an output file named <b>nohit.accession</b>.
```
grep "No hit" mitofish.genes | awk -F "#" '{print $1}' | sed "s/$/.[0-9]/g" >nohit.accession
```
We added some regular expression patterns to the accession numbers with no hits so that the grep search is more specific. The output file <b>nohit.accession</b> looks like this:

```
LN610210.[0-9]
LN610216.[0-9]
LN610224.[0-9]
LN610229.[0-9]
LN610230.[0-9]
LN610231.[0-9]
LN610232.[0-9]
LN610234.[0-9]
LN610235.[0-9]
```
We will also extract the hits from the python search for later use. Output file is <b>hit.list</b>:
```
grep -v "No hit" mitofish.genes >hit.list
```

Next, split up the reference file (<b>nt.genenames</b> generated in previous step) into smaller chunks of 1 million lines each. Each split file will have the prefix x followed by a number e.g. x00, x01

```
cd NCBI
split -d -l 1000000 nt.genenames
cd ..
```

For each accession number with no hit, search each chunk of nt.genenames for matches using all processors (-P 0). -n 1 specifies the number of argument to pass per command line. This takes ~39 hours to run for 144,308 accession numbers.

```
for id in $(cat nohit.accession)
do
string=`ls NCBI/x* | xargs -n 1 -P 0 grep $id`
echo "$id%$string" >>nohit.genes
done
```
Output file is <b>nohit.genes</b>. First, let's clean up the output file by removing non-specific matches and removing the regular expression patterns following the accession numbers:

```
grep "#" nohit.genes | sed "s/.\[0\-9\]//g" >nohit.genes.clean
```

Now, combine <b>hit.list</b> (accession numbers with exact matches with NCBI accession numbers) with <b>nohit.genes.clean</b> (accession numbers with duplicated sequences). The output file will be <b>mitofish.hit.list</b>:
```
cat hit.list nohit.genes.clean >mitofish.hit.list
```


In [8]:
# From a user-provided list of genera/species/subspecies, this script extracts the corresponding GenBank accession numbers 
# and gene names of their 12S rRNA sequences or mitochondrial sequences, if available.

# This script is interactive and takes 3 inputs, in the following order:
# 1) Input file name with extension (e.g. input.txt)
# 2) Output file prefix for output files: <prefix>_genus.hits.txt, <prefix>_species.hits.txt, <prefix>_fulltaxonomy.hits.txt
# 3) Reference database - either 12S.list or mitofish.hit.list
# Output files are separated by "#" in the format: GenBank_Accession#Gene_description


import sys
import os.path
from os import path

# Modify input file name here
input_file = input()
ref=tuple(open(input_file,'r'))


output_prefix = input()

full_path=str(output_prefix+"_fulltaxonomy.hits.txt")
species_path=str(output_prefix+"_species.hits.txt")
genus_path=str(output_prefix+"_genus.hits.txt")

# Throw an error message and exit if output file(s) already exist

if path.exists(full_path) or path.exists(species_path) or path.exists(genus_path) :
    sys.exit("Error: Output file exists! Please rename output file and try again!")

# Create output files if all is well
                  
full_hit=open(full_path,'a')
species_hit = open(species_path,'a')
genus_hit = open(genus_path,'a')

# Create sets to hold lines already seen, for line deduplication later
full_seen=set()
species_seen=set()
genus_seen=set()

reference_file = input()
    
i = 0
while (i < len(ref)):
    
    taxa=str(ref[i]).rsplit()
    fulltaxa=str(ref[i]).rsplit("\n")
    fullquery=str(fulltaxa[0])
    squery=str(taxa[0]+" "+taxa[1])
    gquery=str(taxa[0])
    
    fullcount=0
    scount=0
    gcount=0
    
    qcount = i+1
    print ("=== Searching query #%d ===" % qcount)
    
    with open(reference_file, 'r') as f:
        for line in f.readlines():
            if fullquery in line:
                fullcount += 1
                if line not in full_seen:
                     full_hit.write(line)
                full_seen.add(line)
            if squery in line:
                scount += 1
                if line not in species_seen:
                     species_hit.write(line)
                species_seen.add(line)
            if gquery in line:
                gcount += 1 
                if line not in genus_seen:
                     genus_hit.write(line)
                genus_seen.add(line)

    print("%s hits (full taxonomy): %d" % (fullquery,fullcount))
    print("%s hits (species): %d" % (squery,scount))
    print("%s hits (genus): %d" % (gquery,gcount))
    
    i += 1

full_hit.close()
species_hit.close()
genus_hit.close()

print ("==== Run complete! ===")
print ("Accession number of full taxonomy hits and description saved in <%s_fulltaxonomy.hits.txt>" % output_prefix)
print ("Accession number of species hits and description saved in <%s_species.hits.txt>" % output_prefix)
print ("Accession number of genus hits and description saved in <%s_genus.hits.txt>" % output_prefix)




subspecies
test
12S.list
=== Searching query #1 ===
Histioteuthis celetaria celetaria hits (full taxonomy): 0
Histioteuthis celetaria hits (species): 0
Histioteuthis hits (genus): 1
=== Searching query #2 ===
Histioteuthis corona corona hits (full taxonomy): 0
Histioteuthis corona hits (species): 0
Histioteuthis hits (genus): 1
=== Searching query #3 ===
Lampadena urophaos atlantica hits (full taxonomy): 0
Lampadena urophaos hits (species): 3
Lampadena hits (genus): 5
=== Searching query #4 ===
Notoscopelus elongatus kroyeri hits (full taxonomy): 0
Notoscopelus elongatus hits (species): 0
Notoscopelus hits (genus): 7
=== Searching query #5 ===
Scopelogadus mizolepis mizolepis hits (full taxonomy): 0
Scopelogadus mizolepis hits (species): 0
Scopelogadus hits (genus): 5
=== Searching query #6 ===
Stomias boa boa hits (full taxonomy): 0
Stomias boa hits (species): 2
Stomias hits (genus): 11
==== Run complete! ===
Accession number of full taxonomy hits and description saved in <test_fullta